In [ ]:
import ipywidgets as w
import plotly.graph_objects as go
import math
import json
from js.models import TraceWidget
from js.gcode import ReaderFactory
from hublib.ui import Submit
import sys 
import os
import importlib
import nanohub.remote as nr
from IPython.display import clear_output
from traitlets import Unicode
import pathlib
import uuid

In [ ]:
class PathSelector(w.Accordion):
    file = Unicode("")
    def __init__(self, **kwargs):
        self.value = os.environ['HOME']
        self.select = w.SelectMultiple(options=['init'], value=(), rows=5, description='')
        super(PathSelector, self).__init__(children=[self.select])
        self.selected_index = None
        self.refresh(self.value)
        self.select.observe(self.on_update, 'value')
        super()

    def on_update(self, change):
        if len(change['new']) > 0:
            self.refresh(change['new'][0])

    def refresh(self, item):
        path = os.path.abspath(os.path.join(self.value, item))

        if os.path.isfile(path):
            self.set_title(0, path)
            self.file = path
            self.selected_index = None

        else:  # os.path.isdir(path)
            self.value = path

            # Build list of files and dirs
            keys = ['[..]']
            for item in os.listdir(path):
                if os.path.isdir(os.path.join(path, item)):
                    keys.append('[' + item + ']')
                else:
                    iteml = pathlib.Path(item).suffix.lower()
                    if (iteml in [".gcode",".nc",".es", ".inp"]): 
                        keys.append(item)

            # Sort and create list of output values
            keys.sort(key=str.lower)
            vals = []
            for k in keys:
                if k[0] == '[':
                    vals.append(k[1:-1])  # strip off brackets
                else:
                    vals.append(k)

            # Update widget
            self.set_title(0, "Project_GCodes")
            self.select.options = list(zip(keys, vals))
            with self.select.hold_trait_notifications():
                self.select.value = ()


In [ ]:
def integral_VS(parameters,t,dt,T,dT):
    C1    = parameters[0];
    C2    = parameters[1];
    C3    = parameters[2];
    Tg    = parameters[3];
    Tc    = parameters[4];
    Tmelt = parameters[5];
    n     = parameters[6];

    ## Time and temperature at the start and end of increment
    T_mid = T+(dT/2.0);
    T1 = T;
    T2 = T1+dT;
    t1=t;
    t2=t1+dt;

    ## Compute integral from t to t+dt
    Term1_1 = - C2 / (T1-Tg+Tc);
    Term1_2 = - C2 / (T2-Tg+Tc);

    Term2_1 = - C3/(T1*math.pow((Tmelt-T1),2));
    Term2_2 = - C3/(T2*math.pow((Tmelt-T1),2));

    F_1=T1 * math.exp(Term1_1+Term2_1)*n*(math.pow(t1,(n-1)));
    F_2=T2 * math.exp(Term1_2+Term2_2)*n*(math.pow(t2,(n-1)));

    dI = C1*((F_1+F_2)/2.0)*dt;
    return dI;

In [ ]:
def linspace(a,b,n):
    n = math.floor(n)
    if(n<2):
        if n == 1:
            return[a]
        else:
            return[]
    l = list(range(n))
    #l.reverse()
    return [(i*b+(n-i-1)*a)/n for i in l]    


In [ ]:
##
# This function returns the increment in melting computed with th melting model by Greco and Maffezzoli
# Compute incremental change in crystallinity due to melting 
# @melting_GM
# @author: denphi, denphi@denphi.com, Purdue University
# @author: Eduardo Barocio, Purdue University
# @param {Kmb} a float, 
# @param {d} a float,
# @param {Tc} a float,
# @param {T} a float, initial temperature
# @param {dT} a float, delta of temperature
# @return  {dX}
##

def melting_GM(Kmb, d, Tc, T, dT):  
    T_mid=T+ dT/2.0;
    Melt1 = d/(1.0-d);
    Melt2 = Kmb * (T_mid-Tc);
    dX_dT = Kmb * math.exp(-Melt2)*math.pow(1+(d-1)*math.exp(-Melt2),Melt1);
    dX = dX_dT * dT;
    return dX;

In [ ]:
def meltingKinetics(db, E_Temp, S_Temp, units, temp):
    if (E_Temp>S_Temp):
        #Heating rate (K/min)
        Heating_Rate = 50.0;
        #Inferred inputs:
        #Start and end temperatures (K)

        t_inc= 1.0;
        X_ini=0.84;

        Kmb = db["melting"]["xkmb"]
        d = db["melting"]["d"]
        Tc = db["melting"]["Tc"]

        X_inft=0.84;

        Total_time = 60.0*(E_Temp-S_Temp)/Heating_Rate;
        steps=math.floor(Total_time/t_inc);
        time = linspace(0,Total_time,steps);
        Temp = [S_Temp + t*(Heating_Rate/60.0) for t in time] 
        dt = [time[ii+1] - time[ii] for ii in range(steps-1)]
        dT = [Temp[ii+1] - Temp[ii] for ii in range(steps-1)]
        X=[0 for ii in range(steps)]
        X[0] = X_ini;

        for ii in range(steps-1):
            #Calculate time and temperature increments
            t1 = time[ii];
            t2 = t1 + dt[ii];
            T1 = Temp[ii];
            T2 = T1 + dT[ii];            
            dX_M = melting_GM(Kmb, d, Tc, T1, dT[ii]); 
            dX_M = dX_M * X_inft;
            X[ii+1]= X[ii] - dX_M;

        if (units == "minutes"):
            time = [t/60.0 for t in time]
        elif (units == "hours"):
            time = [t/(60.0*60.0) for t in time]

        if (temp == "Fahrenheit"):
            Temp = [(t*(9/5)) - 459.67 for t in Temp]
        elif (temp == "Celsius"):
            Temp = [t - 273.15 for t in Temp]
        
        trace1 = {
            'type' : 'scatter',
            'name' : 'Melting',
            'x' : time, 
            'y' : X,
            'line': { 'color' : "#A3A1FB"},
            'yaxis' : 'y',
            'fill' : 'tozeroy'
        }
        trace2 = {
            'type' : 'scatter',
            'name' : 'Temperature',
            'x' : time, 
            'y' : Temp,
            'line': { 'color' : "#849E2A"},
            'yaxis' : 'y2',          
        }

        layout = {        
            'margin': {'l': 45, 'r': 45, 't': 30, 'b': 30},
            'xaxis' : {
                'title' : 'Time (' + units + ')',
            },
            'yaxis' : {
                'title' : 'Melting Kinetics',
                'side' : 'left',
                'titlefont' : { 'color': '#A3A1FB' },
                'tickfont' : { 'color': '#A3A1FB' },

            },
            'yaxis2' : {
                'title' : 'Temperature (' + temp[0] + ')',
                'side' : 'right',
                'overlaying' : 'y',          
                'titlefont' : { 'color': '#849E2A' },
                'tickfont' : { 'color': '#849E2A' },
                'showgrid' : False
            }, 
            'showlegend' : False,

        }            
        return {'layout': layout, 'traces' : [trace1, trace2] }
    else :
        return {'layout': {}, 'traces' : [] }     

In [ ]:
##    
# Creates a Ploty representation of the Crystalization Kinetics Model (developed by Eduardo Barocio - ebarocio@purdue.edu )
# @author: denphi, denphi@denphi.com, Purdue University
# @author: Eduardo Barocio, Purdue University
# @param {material} An string that represents the material to be user on the simulation, 
# @param {E_Temp} An float that represents the inital temperature, 
# @param {S_Temp} An string that represents the material to be user on the simulation, 
# @param {units} Dimension used to create the plotly descriptor, 
# @return {plot} an object representing Plotly plot
##

def CrystalizationKinetics(db, S_Temp, E_Temp, units, temp):
    if (S_Temp > E_Temp):
        Cooling_Rate = -50.0;
        t_inc= 0.2;
        crystal = db["crystallization"]
        crystal['w2'] = 1.0-crystal['w1']
        CK_M_1_Param = [crystal['C11'], crystal['C12'], crystal['C13'], crystal['Tg'], crystal['Tadd1'], crystal['Tm1'], crystal['xn1'], crystal['w1']];
        CK_M_2_Param = [crystal['C21'], crystal['C22'], crystal['C23'], crystal['Tg'], crystal['Tadd2'], crystal['Tm2'], crystal['xn2'], crystal['w2']];
        X_inft = 0.84;
        Total_time = 60.0 * (S_Temp-E_Temp)/(-Cooling_Rate);
        steps = math.floor(Total_time / t_inc);
        time = linspace(0,Total_time,steps);
        Temp = [S_Temp + t*(Cooling_Rate/60.0) for t in time]

        dt = [time[ii+1] - time[ii] for ii in range(steps-1)]
        dT = [Temp[ii+1] - Temp[ii] for ii in range(steps-1)]
        #Initialize variables
        X=[0 for ii in range(steps)]
        X_M1=[0 for ii in range(steps)]
        X_M2=[0 for ii in range(steps)]
        I_M1= 0;
        I_M2= 0;
        #Compute degree of crystallinity 
        for ii in range(steps-1):

            #Calculate time and temperature increments
            t1 = time[ii];
            t2 = t1 + dt[ii];
            T1 = Temp[ii];
            T2 = T1 + dT[ii];        


            ## ********************Crystallization*********************************           
            ##   Crystallization at end of increment
            ##   Compute integrals of individual mechanisms

            dI_M1 = integral_VS(CK_M_1_Param, t1,dt[ii],T1,dT[ii]);
            dI_M2 = integral_VS(CK_M_2_Param, t1,dt[ii],T1,dT[ii]);
            I_M1 = I_M1 + dI_M1;
            I_M2 = I_M2 + dI_M2;
            F_M1 = 1-math.exp(-I_M1);
            F_M2 = 1-math.exp(-I_M2);
            X_M1[ii+1]=X_inft * CK_M_1_Param[7] * F_M1;
            X_M2[ii+1]=X_inft * CK_M_2_Param[7] * F_M2;
            X[ii+1]=X_M1[ii+1] + X_M2[ii+1];    

            ##Compute contribution of each mechanisms used to update Integrals 
            ## C_M1 and C_M2 are not used 
            '''
            if ((I_M1 > 0) && (I_M2 >0)){
               C_M1 = (X_inft * CK_M_1_Param[7] * (1-math.exp(-I_M1))) / X[ii+1];
               C_M2 = 1 - C_M1;
            } else {
               C_M1 = 0.5;
               C_M2 = 0.5;
            }
            '''
    
        if (units == "minutes"):
            time = [t/60.0 for t in time]
        elif (units == "hours"):
            time = [t/(60.0*60.0) for t in time]

        if (temp == "Fahrenheit"):
            Temp = [(t*(9.0/5.0)) - 459.67 for t in Temp]
        elif (temp == "Celsius"):
            Temp = [t - 273.15 for t in Temp]

        trace1 = {
            'type' : 'scatter',
            'name' : 'Crystallinity',
            'x' : time, 
            'y' : X,
            'line': { 'color' : "#A3A1FB"},
            'yaxis' : 'y',
            'fill' : 'tozeroy'

        }
        trace2 = {
            'type' : 'scatter',
            'name' : 'Temperature',
            'x' : time, 
            'y' : Temp,
            'line': { 'color' : "#849E2A"},
            'yaxis' : 'y2',          
        }

        layout = {        
            'margin': {'l': 45, 'r': 45, 't': 30, 'b': 30},
            'xaxis' : {
                'title' : 'Time (' + units + ')',
            },
            'yaxis' : {
                'title' : 'Crystallinity - X',
                'side' : 'left',
                'titlefont' : { 'color': '#A3A1FB' },
                'tickfont' : { 'color': '#A3A1FB' },

            },
            'yaxis2' : {
                'title' : 'Temperature (' + str(temp[0]) + ')',
                'side' : 'right',
                'overlaying' : 'y',          
                'titlefont' : { 'color': '#849E2A' },
                'tickfont' : { 'color': '#849E2A' },
                'showgrid' : False
            }, 
            'showlegend' : False,

        }            
        return {'layout': layout, 'traces' : [trace1, trace2] }    
    else :
        return {'layout': {}, 'traces' : [] }            

In [ ]:
#sys.path.append('/home/cdmhub/denphi/Additive3D/additive3D/') 
#from a3d.api import a3d_api 
#from a3d.vtk.vtk2model import vtk2model

class a3dModules():
    def __init__(self):
        self.path = None
        self.reloadModules()
        
    def reloadModules(self):
        if self.path is None:
            self.a3d_api = None
            self.a3d_gcode = None
            self.vtk2model = None
            if "a3d_api" in sys.modules:
                del sys.modules["a3d_api"]
            if "a3d.api" in sys.modules:
                del sys.modules["a3d.api"]
            if "vtk2model" in sys.modules:
                del sys.modules["vtk2model"]
        else:
            spec = importlib.util.spec_from_file_location("a3d_api", os.path.join(self.path,"a3d","api","__init__.py"))
            module = importlib.util.module_from_spec(spec)
            sys.modules[spec.name] = module 
            sys.modules["a3d.api"] = module 
            spec.loader.exec_module(module)
            if(self.a3d_api is None):
                self.a3d_api = importlib.import_module("a3d_api.a3d_api")
            self.a3d_api = importlib.reload(self.a3d_api)
            
            if(self.a3d_gcode is None):
                self.a3d_gcode = importlib.import_module("a3d_api.new_gcode_class")
            self.a3d_gcode = importlib.reload(self.a3d_gcode)

            spec2 = importlib.util.spec_from_file_location("vtk2model", os.path.join(self.path,"a3d","vtk","vtk2model","vtk2model.py"))
            module2 = importlib.util.module_from_spec(spec2)
            sys.modules[spec2.name] = module2
            spec2.loader.exec_module(module2)
            self.vtk2model = module2

class Wizard(w.Tab):
    def __init__(self, **kwargs):
        self.a3d_model = None
        self.projects = self._set_projects()            
        self.path = os.getcwd()
        self.slicingc = self.buildSlicing()
        self.materialc = self.buildMaterial()
        self.verifyc = self.buildVerify()
        self.debug = w.Output()
        self.modules = a3dModules()
        self.project.observe(lambda c, s=self: Wizard._handle_project(s), names='value')
        self._handle_project();
        super(Wizard, self).__init__(children=[self.slicingc, self.materialc, self.verifyc, self.debug])
        self.set_title(0, "Slicing and Machine")
        self.set_title(1, "Material and Process")
        self.set_title(2, "Verify and Submit")
        self.set_title(3, "Log")
        self.updatePlotMaterial()

    def _set_projects(self):
        auth_data = {
            'grant_type' : 'tool',
        }
        with open(os.environ["SESSIONDIR"]+"/resources") as file:
            lines = [line.split(" ", 1) for line in file.readlines()]
            properties = {line[0].strip(): line[1].strip() for line in lines if len(line)==2}
            auth_data["sessiontoken"] = properties["session_token"]
            auth_data["sessionnum"] = properties["sessionid"]
        session = nr.Session(auth_data, url='https://cdmhub.org/api/')
        req_json = session.requestGet('/projects/list?verbose=1')
        req_json = req_json.json()
        projects = {}
        for p in req_json['projects']:
            #if 'a3dconsortium' in p['alias']:
            if 'cams' in p['alias']:
                path = '/data/projects/' + p['alias'] + "/files"
                if os.path.exists(path):
                    p['path'] = path
                    projects[p['alias']] = p
        return projects
                    
                    
    def _handle_change(self, change):
        with self.debug:
            self.json_card.value = "<pre>" + self.buildJson( ) + "</pre>"

    def _handle_project(self):
        with self.debug:
            self.modules.path = os.path.join(self.projects[self.project.value]['path'],'a3d')
            self.modules.reloadModules()
            cardnames = self.modules.a3d_gcode.get_card_names()
            self.printer.options = cardnames['machine']
            self.material.options = cardnames['material']
            self.a3d_model = self.modules.a3d_api.A3DModel('dummy')
            
            codes_path = os.path.join(self.projects[self.project.value]['path'],'Project_GCodes')
            if not os.path.exists(codes_path):
                os.makedirs(codes_path)
            self.path.value = codes_path
            self.path.file = ""
            self.path.refresh(self.path.value)

        
    def _handle_default(self, change):
        codes_path = os.path.join(self.projects[self.project.value]['path'],'Project_GCodes')
        if not os.path.exists(codes_path):
            os.makedirs(codes_path)
        self.path.value = codes_path
        self.path.file = ""
        self.path.refresh(self.path.value)
        
        name = os.path.join(codes_path,"demo_flat.es")
        with open(name, "wt") as fp:
            fp.write('\n'.join(["0.0,0,0,0,0,3","0.1197,9.225,15.4,0.0,1,3","1.9352,190.775,15.4,0.0,1,3","1.9967,190.775,21.55,0.0,1,3","3.8122,9.225,21.55,0.0,1,3","3.8737,9.225,27.7,0.0,1,3","5.6892,190.775,27.7,0.0,1,3","5.7507,190.775,33.85,0.0,1,3","7.5662,9.225,33.85,0.0,1,3","7.6277,9.225,40.0,0.0,1,3","9.4432,190.775,40.0,0.0,1,3","9.5047,190.775,46.15,0.0,1,3","11.3202,9.225,46.15,0.0,1,3","11.3817,9.225,52.3,0.0,1,3","13.1972,190.775,52.3,0.0,1,3","13.2587,190.775,58.45,0.0,1,3","15.0742,9.225,58.45,0.0,1,3","15.1357,9.225,64.6,0.0,1,3","16.9512,190.775,64.6,0.0,1,3","17.0127,190.775,70.75,0.0,1,3","18.8282,9.225,70.75,0.0,1,3","18.8897,9.225,76.9,0.0,1,3","20.7052,190.775,76.9,0.0,1,3","20.7667,190.775,83.05,0.0,1,3","22.5822,9.225,83.05,0.0,1,3","22.6437,9.225,89.2,0.0,1,3","24.4592,190.775,89.2,0.0,1,3","24.5207,190.775,95.35,0.0,1,3","26.3362,9.225,95.35,0.0,1,3","26.3977,9.225,101.5,0.0,1,3","28.2132,190.775,101.5,0.0,1,3","28.2747,190.775,107.65,0.0,1,3","30.0902,9.225,107.65,0.0,1,3","30.1517,9.225,113.8,0.0,1,3","31.9672,190.775,113.8,0.0,1,3","32.0287,190.775,119.95,0.0,1,3","33.8442,9.225,119.95,0.0,1,3","33.9057,9.225,126.1,0.0,1,3","35.7212,190.775,126.1,0.0,1,3","35.7827,190.775,132.25,0.0,1,3","37.5982,9.225,132.25,0.0,1,3","37.6597,9.225,138.4,0.0,1,3","39.4752,190.775,138.4,0.0,1,3","39.5367,190.775,144.55,0.0,1,3","41.3522,9.225,144.55,0.0,1,3","41.4137,9.225,150.7,0.0,1,3","43.2292,190.775,150.7,0.0,1,3","43.2907,190.775,156.85,0.0,1,3","45.1062,9.225,156.85,0.0,1,3","45.1677,9.225,163.0,0.0,1,3","46.9832,190.775,163.0,0.0,1,3","47.0447,190.775,169.15,0.0,1,3","48.8602,9.225,169.15,0.0,1,3","48.9217,9.225,175.3,0.0,1,3","50.7372,190.775,175.3,0.0,1,3","50.7987,190.775,181.45,0.0,1,3","52.6142,9.225,181.45,0.0,1,3","52.6757,9.225,187.6,0.0,1,3","54.5321,190.775,187.6,1.5,1,3","56.3476,9.225,187.6,1.5,1,3","56.4091,9.225,181.45,1.5,1,3","58.2246,190.775,181.45,1.5,1,3","58.2861,190.775,175.3,1.5,1,3","60.1016,9.225,175.3,1.5,1,3","60.1631,9.225,169.15,1.5,1,3","61.9786,190.775,169.15,1.5,1,3","62.0401,190.775,163.0,1.5,1,3","63.8556,9.225,163.0,1.5,1,3","63.9171,9.225,156.85,1.5,1,3","65.7326,190.775,156.85,1.5,1,3","65.7941,190.775,150.7,1.5,1,3","67.6096,9.225,150.7,1.5,1,3","67.6711,9.225,144.55,1.5,1,3","69.4866,190.775,144.55,1.5,1,3","69.5481,190.775,138.4,1.5,1,3","71.3636,9.225,138.4,1.5,1,3","71.4251,9.225,132.25,1.5,1,3","73.2406,190.775,132.25,1.5,1,3","73.3021,190.775,126.1,1.5,1,3","75.1176,9.225,126.1,1.5,1,3","75.1791,9.225,119.95,1.5,1,3","76.9946,190.775,119.95,1.5,1,3","77.0561,190.775,113.8,1.5,1,3","78.8716,9.225,113.8,1.5,1,3","78.9331,9.225,107.65,1.5,1,3","80.7486,190.775,107.65,1.5,1,3","80.8101,190.775,101.5,1.5,1,3","82.6256,9.225,101.5,1.5,1,3","82.6871,9.225,95.35,1.5,1,3","84.5026,190.775,95.35,1.5,1,3","84.5641,190.775,89.2,1.5,1,3","86.3796,9.225,89.2,1.5,1,3","86.4411,9.225,83.05,1.5,1,3","88.2566,190.775,83.05,1.5,1,3","88.3181,190.775,76.9,1.5,1,3","90.1336,9.225,76.9,1.5,1,3","90.1951,9.225,70.75,1.5,1,3","92.0106,190.775,70.75,1.5,1,3","92.0721,190.775,64.6,1.5,1,3","93.8876,9.225,64.6,1.5,1,3","93.9491,9.225,58.45,1.5,1,3","95.7646,190.775,58.45,1.5,1,3","95.8261,190.775,52.3,1.5,1,3","97.6416,9.225,52.3,1.5,1,3","97.7031,9.225,46.15,1.5,1,3","99.5186,190.775,46.15,1.5,1,3","99.5801,190.775,40.0,1.5,1,3","101.3956,9.225,40.0,1.5,1,3","101.4571,9.225,33.85,1.5,1,3","103.2726,190.775,33.85,1.5,1,3","103.3341,190.775,27.7,1.5,1,3","105.1496,9.225,27.7,1.5,1,3","105.2111,9.225,21.55,1.5,1,3","107.0266,190.775,21.55,1.5,1,3","107.0881,190.775,15.4,1.5,1,3"]))
        self.path.file = name
        self.path.refresh(self.path.value)
        self.path.set_title(0,self.path.file)
        self._handle_change(change)
                
    def _handle_upload(self, change):
        with self.debug:
            self.ts = ""
            if self.upload._counter > 0 :
                key_file = list(self.upload.value.keys())[0]
                uploaded_file = self.upload.value[key_file]
                item = uploaded_file['metadata']['name']
                iteml = pathlib.Path(item).suffix.lower()
                if (iteml in [".gcode",".nc",".es", ".inp"]): 
                    codes_path = os.path.join(self.projects[self.project.value]['path'],'Project_GCodes')
                    if not os.path.exists(codes_path):
                        os.makedirs(codes_path)
                    self.path.value = codes_path
                    self.path.file = ""
                    self.path.refresh(self.path.value)

                    name = os.path.join(codes_path,uploaded_file['metadata']['name'])
                    with open(name, "wb") as fp:
                        fp.write(uploaded_file['content'])

                    self.path.value = os.getcwd()
                    self.path.file = name
                    self.path.refresh(self.path.value)
                    self.path.set_title(0,self.path.file)
                else : 
                    self.path.value = os.path.join(self.projects[self.project.value]['path'],'Project_GCodes')
                    self.path.file = ""
                    self.path.refresh(self.path.value)
                    self.path.set_title(0, "Project_GCodes")
                self.upload = w.FileUpload(
                    accept = ".gcode,.nc,.ts",
                    description = "Upload Gcode (small files)", 
                    multiple=False, 
                    layout=w.Layout(width='auto'),
                )
                self.upload.observe(lambda c, s=self: Wizard._handle_upload(s, c), names='value')
                self.files.children = [self.upload,self.default,self.path]                    
            self._handle_change(change)
            
    def _handle_file(self, change):
        with self.debug:
            self.ts = ""
            if self.path.file != "" :
                gci = self.modules.a3d_gcode.GcodeInterpreter(self.path.file, self.printer.value)
                gci.es_builder()
                bwidth, bheight = gci.get_bead_dims()
                if bheight == None:
                    bheight = self.tU(1.5, "length")
                if bwidth == None:
                    bwidth = self.tU(6.15, "length")
                self.bead_height.value = bheight
                self.bead_width.value = bwidth
                es = gci.get_event_series()
                self.ts = es.print_event_series()
                if len (self.ts) > 0:
                    self.trace.lines = self.ts.splitlines()
            self._handle_change(change)
            
    def _handle_fidelity(self, change):
        if self.level_of_fidelity.value == "Low":
            self.cooling.value = 120
            self.bead_per_height.value = 1
            self.bead_per_width.value = 1
            self.deposition.value = 25
            self.cooling.disabled = True
            self.bead_per_height.disabled = True
            self.bead_per_width.disabled = True
            self.deposition.disabled = True
        elif self.level_of_fidelity.value == "Medium":
            self.cooling.value = 60
            self.bead_per_height.value = 1 
            self.bead_per_width.value = 2
            self.deposition.value = 5
            self.cooling.disabled = True
            self.bead_per_height.disabled = True
            self.bead_per_width.disabled = True
            self.deposition.disabled = True
        elif self.level_of_fidelity.value == "High":
            self.cooling.value = 30
            self.bead_per_height.value = 2
            self.bead_per_width.value = 2
            self.deposition.value = 1
            self.cooling.disabled = True
            self.bead_per_height.disabled = True
            self.bead_per_width.disabled = True
            self.deposition.disabled = True
        else:
            self.cooling.disabled = False
            self.bead_per_height.disabled = False
            self.bead_per_width.disabled = False
            self.deposition.disabled = False
        self._handle_change(change)
            

    def _handle_path(self, change):
        self.trace.field = self.toolpath_options.value

    def _handle_color(self, change):
        self.trace.colormap = self.color.value

    def buildSlicing(self):
        projects = [(v['title'],k) for k,v in self.projects.items()]
        self.project = w.Dropdown(
            description = "Select Project", 
            style={'description_width': 'initial'}, 
            layout=w.Layout(width='auto'),
            options = projects
        )
        
        self.session = w.Text(
            description = "Simulation_ID", 
            style={'description_width': 'initial'}, 
            layout=w.Layout(width='auto'),
            value = "additive3d_" + str(uuid.uuid4()).replace("-", "_"),
            disabled = False
        )
            
        label1 = w.HTML(
            value="<h3>Simulation Title</h3>"
        )
        self.title = w.Text(
            description = "Simulation Title", 
            style={'description_width': 'initial'}, 
            layout=w.Layout(width='auto'),
            value = "Flatplate job"
        )
        self.title.observe(lambda c, s=self: Wizard._handle_change(s, c), names='value')

        label2 = w.HTML(
            value="<h3>Gcode/Event Series</h3>"
        )
        #printers = ["CAMRI", "BAAM", "LSAM"]
        self.printer = w.Dropdown(
            description = "Select Printer", 
            style={'description_width': 'initial'}, 
            layout=w.Layout(width='auto'),
            #options = printers,
            #value = printers[0]
        )
        self.printer.observe(lambda c, s=self: Wizard._handle_change(s, c), names='value')
        self.upload = w.FileUpload(
            description = "Upload Gcode (small files)", 
            multiple=False, 
            layout=w.Layout(width='auto')
        )
        self.upload.observe(lambda c, s=self: Wizard._handle_upload(s, c), names='value')

        self.default = w.Button(
            description = "Load Demo Event Series", 
            layout=w.Layout(width='auto')
        )
        self.default.on_click(lambda c, s=self: Wizard._handle_default(s, c))
        
        self.path = PathSelector(layout=w.Layout(padding='3px'))
        self.path.observe(lambda c, s=self: Wizard._handle_file(s, c), names='file')

        label3 = w.HTML(
            value="<h3>Bead Dimensions (units)</h3>"
        )
        fidelity_options = ["Low", "Medium", "High", "Custom"]
        self.level_of_fidelity = w.Dropdown(
            description = "LevelOfFidelity", 
            style={'description_width': 'initial'}, 
            layout=w.Layout(width='auto'),
            options=fidelity_options,
            value=fidelity_options[0]
        )
        self.level_of_fidelity.observe(lambda c, s=self: Wizard._handle_fidelity(s, c), names='value')
        self.cooling = w.BoundedIntText(
            description = "Cooling time increment (sec)", 
            style={'description_width': 'initial'}, 
            layout=w.Layout(width='auto'), 
            value=self.tU(120.0, "time"), 
            min=0,
            max=1000
        )
        self.cooling.observe(lambda c, s=self: Wizard._handle_change(s, c), names='value')

        self.bead_width = w.BoundedFloatText(
            description = "Width (mm)", 
            style={'description_width': 'initial'}, 
            layout=w.Layout(width='auto'), 
            value=self.tU(6.15, "length"), 
            min=0,
            max=1000
        )
        self.bead_width.observe(lambda c, s=self: Wizard._handle_change(s, c), names='value')

        self.bead_height = w.BoundedFloatText(
            description = "Height (mm)", 
            style={'description_width': 'initial'}, 
            layout=w.Layout(width='auto'), 
            value=self.tU(1.5, "length"), 
            min=0,
            max=1000
        )
        self.bead_height.observe(lambda c, s=self: Wizard._handle_change(s, c), names='value')

        self.bead_per_width = w.BoundedIntText(
            description = "Elements per bead width (sec)", 
            style={'description_width': 'initial'}, 
            layout=w.Layout(width='auto'), 
            value=1, 
            disabled = True,
            min=0,
            max=100
        )
        self.bead_per_width.observe(lambda c, s=self: Wizard._handle_change(s, c), names='value')

        self.bead_per_height = w.BoundedIntText(
            description = "Elements per bead heigth (sec)", 
            style={'description_width': 'initial'}, 
            layout=w.Layout(width='auto'), 
            value=1, 
            disabled = True,
            min=0,
            max=100
        )
        self.bead_per_height.observe(lambda c, s=self: Wizard._handle_change(s, c), names='value')

        self.deposition = w.BoundedIntText(
            description = "Deposition time increment (sec)",
            style={'description_width': 'initial'}, 
            layout=w.Layout(width='auto'), 
            value=self.tU(25.0, "time"), 
            disabled = True,
            min=0,
            max=100
        )
        self.deposition.observe(lambda c, s=self: Wizard._handle_change(s, c), names='value')
        
        label4 = w.HTML(
            value="<h3>Instructions</h3>"
        )
        self.instructions = w.HTML(
            value="Drag the gcode or timeseries file to the Drag & Drop Area, and select a proper printer"
        )
        label5 = w.HTML(
            value="<h3>Toolpath Visualization</h3>"
        )
        toolpaths = [("Activation Time",0), ("Layer Time",4), ("Z Height",3)]
        self.toolpath_options = w.Dropdown(
            description = "Toolpath Options:", 
            style={'description_width': 'initial'}, 
            layout=w.Layout(width='auto'),
            options=toolpaths,
            value = 0
        )
        self.toolpath_options.observe(lambda c, s=self: Wizard._handle_path(s, c), names='value')

        palette_options = ["blackbody","greys","blues"]
        self.color = w.Dropdown(
            description = "Color :", 
            style={'description_width': 'initial'}, 
            layout=w.Layout(width='auto'),
            options=palette_options,
            value=palette_options[0]
        )
        self.color.observe(lambda c, s=self: Wizard._handle_color(s, c), names='value')

        self.model = w.Output()
        label6 = w.HTML(
            value="<h3>Toolpath Details</h3>"
        )
        
        self.ts = []
        self.trace = TraceWidget(
            colormap=self.color.value, 
            field=self.toolpath_options.value, 
            lines=self.ts
        )
        
        self.dimensions = w.HTML(value="DIMENSION:[0.00 0.00 0.00]")
        self.total_time = w.HTML(value="TOTAL TIME:[0.00 0.00 0.00]")
        self.max_time = w.HTML(value="MAX TIME LAYER:[0.00 0.00 0.00]")
        self.files = w.HBox([
            self.upload,
            self.default,
            self.path
        ], layout=w.Layout(flex="1 1", padding="10px"))
        
        return w.HBox([
            w.VBox([
                label1, 
                self.title, 
                self.session,
                self.project,
                label2, 
                self.printer, 
                self.files,
                label3,
                self.bead_width,
                self.bead_height,
                self.level_of_fidelity,
                self.cooling,
                self.bead_per_width,
                self.bead_per_height,
                self.deposition
            ], layout=w.Layout(flex="1 1", padding="4px")), 
            w.VBox([
                label4, 
                self.instructions,
                label5,
                self.toolpath_options,
                self.color,
                self.model,
                self.trace,
                label6,
                self.dimensions,
                self.total_time,
                self.max_time
            ], layout=w.Layout(flex="1 1", padding="4px"))
        ])

    
    def buildMaterial(self):
        label1 = w.HTML(value="<h3>Material and EDAM Process</h3>")
        self.material = w.Dropdown(
            description = "Material :", 
            style={'description_width': 'initial'}, 
            layout=w.Layout(width='auto')
            #options = material_options,
            #value = material_options[0]
        )
        self.material.observe(lambda c, s=self: Wizard._handle_change(s, c), names='value')
        self.material.observe(lambda c, s=self: Wizard.updatePlotMaterial(s), names='value')

        label2 = w.HTML(value="<h3>Process Conditions</h3>")
        self.extrusionTemp = w.BoundedFloatText(
            description = "Extrusion Temperature (K) :", 
            style={'description_width': 'initial'}, 
            layout=w.Layout(width='auto'), 
            value=self.tU(573.15, "temp"), 
            min = 0,
            max = 1000
        )
        self.extrusionTemp.observe(lambda c, s=self: Wizard._handle_change(s, c), names='value')

        self.plateTemp = w.BoundedFloatText(
            description = "Build Plate Temperature (K) :", 
            style={'description_width': 'initial'}, 
            layout=w.Layout(width='auto'),
            value=self.tU(453.15, "temp"), 
            min = 0,
            max = 1000
        )
        self.plateTemp.observe(lambda c, s=self: Wizard._handle_change(s, c), names='value')

        label3 = w.HTML(value="<h3>Cooling Conditions</h3>")
        self.time_on = w.BoundedFloatText(
            description = "Time to cool on the plate (sec) :", 
            style={'description_width': 'initial'}, 
            layout=w.Layout(width='auto'), 
            value=self.tU(1200.0, "time"),
            min=0,
            max=10000
        )
        self.time_on.observe(lambda c, s=self: Wizard._handle_change(s, c), names='value')

        self.time_off = w.BoundedFloatText(
            description = "Time to cool off the build plate (sec) :", 
            style={'description_width': 'initial'}, 
            layout=w.Layout(width='auto'), 
            value=self.tU(600.0, "time"), 
            min=0,
            max=10000
        )
        self.time_off.observe(lambda c, s=self: Wizard._handle_change(s, c), names='value')

        self.ambient = w.BoundedFloatText(
            description = "Ambient Temperature (K) :", 
            style={'description_width': 'initial'}, 
            layout=w.Layout(width='auto'), 
            value=self.tU(308.15, "temp"), 
            min=0,
            max=10000
        )
        self.ambient.observe(lambda c, s=self: Wizard._handle_change(s, c), names='value')

        label4 = w.HTML(value="<h3>Material Properties</h3>")
        properties = ["Crystalization Kinetics","Melting Kinetics"]
        self.propertyOutput = w.Dropdown(
            description = "Material Property:", 
            style={'description_width': 'initial'}, 
            layout=w.Layout(width='auto'),
            options = properties,
            value = properties[0]
        )
        self.propertyOutput.observe(lambda e, s=self:s.updatePlotMaterial(),"value")
        self.plot = go.FigureWidget()
        self.unitsOutput = w.Text(value="seconds")
        self.tempOutput = w.Text(value="Kelvin")
        return w.HBox([
            w.VBox([
                label1, 
                self.material,
                label2,
                self.extrusionTemp,
                self.plateTemp,
                label3, 
                self.time_on,
                self.time_off,
                self.ambient
            ], layout=w.Layout(flex="1 1", padding="4px")),
            w.VBox([
                label4,
                self.propertyOutput,
                self.plot               
            ], layout=w.Layout(flex="1 1", padding="4px"))
        ])

    def buildJson(self, complete = False):
        ts = "..."
        if complete:
            ts = self.ts
        return json.dumps({
            "job_name": self.title.value,
            "event_series": ts,
            "material":  self.material.value,
            "machine": self.printer.value,
            "job": {
                "build_plate_temperature": self.rU(self.plateTemp.value, "temp"),
                "ambient_temperature": self.rU(self.ambient.value, "temp"),
                "extrudate_temperature": self.rU(self.extrusionTemp.value, "temp"),
                "bead_width": self.rU(self.bead_width.value, "length"),
                "bead_height": self.rU(self.bead_height.value, "length"),
                "elements_per_bead_width": self.bead_per_width.value,
                "elements_per_bead_height": self.bead_per_height.value,
                "step_options": {
                    "initial_increment_time": self.rU(self.deposition.value, "time"),
                    "max_increment_time": self.rU(self.deposition.value, "time"),
                    "output_each_layer": True,
                    "output_each_increment": True,
                    "cooling_time_on_bed": self.rU(self.time_on.value, "time"),
                    "cooling_time_off_bed": self.rU(self.time_off.value, "time"),
                    "cooling_increment": self.rU(self.cooling.value, "time")
                }
            }
        }, indent=2)
     
    def buildVerify(self):
        self.cores = w.BoundedIntText(
            description = "Cpu cores:", 
            style={'description_width': 'initial'}, 
            layout=w.Layout(width='auto'), 
            value=20, 
            min=0,
            max=60
        )
        self.walltime = w.BoundedIntText(
            description = "Walltime:", 
            style={'description_width': 'initial'}, 
            layout=w.Layout(width='auto'), 
            value=60, 
            min=30,
            max=240
        )
        venues_options = ["cmsc@halstead","standby@halstead"]
        self.venue = w.Dropdown(
            description = "Venues :", 
            style={'description_width': 'initial'}, 
            layout=w.Layout(width='auto'),
            options=venues_options,
            value=venues_options[0]
        )
        
        self.json_card = w.HTML(value= "<pre>" + self.buildJson( ) + "</pre>") 
        self.submit = Submit(
            start_func=(lambda c, s=self: Wizard.submitStart(s, c)), 
            done_func=(lambda c, s=self: Wizard.submitEnd(s, c))
        )
        return w.HBox([
            w.VBox([
                self.venue,
                self.cores,
                self.walltime
            ], layout=w.Layout(flex="1 1", padding="4px")),
            w.VBox([
                self.json_card,
                self.submit.w
            ], layout=w.Layout(flex="1 1", padding="4px"))
        ])
    
    def updatePlotMaterial(self):
        data = {'traces' : [], 'layout' : {}}
        with self.debug:
            if self.a3d_model is not None:
                card = self.a3d_model._generate_card(self.material.value)

                if (self.propertyOutput.value == "Melting Kinetics"):     
                    data = meltingKinetics(
                        card,
                        self.rU(self.extrusionTemp.value, "temp"), 
                        self.rU(self.plateTemp.value, "temp"),
                        self.unitsOutput.value,
                        self.tempOutput.value
                    )
                    self.plot.data = []
                    for t in data['traces']:
                        self.plot.add_trace(t)
                    self.plot.layout = data['layout']

                elif (self.propertyOutput.value == "Crystalization Kinetics"):      
                    data = CrystalizationKinetics(
                        card,
                        self.rU(self.extrusionTemp.value, "temp"), 
                        self.rU(self.plateTemp.value, "temp"),
                        self.unitsOutput.value,
                        self.tempOutput.value
                    )
                    self.plot.data = []
                    for t in data['traces']:
                        self.plot.add_trace(t)
                    self.plot.layout = data['layout']

    def submitStart(self, s):
        self.selected_index = 3
        with self.debug:
            #clear_output()
            session_id = self.session.value
            session_path = self.projects[self.project.value]['path']
            session_name = 'sim_data'
            os.chdir(session_path)
            if not os.path.exists(session_id):
                os.makedirs(session_id)
            os.chdir(session_path + "/" + session_id)
            with open('sim_data_card.json', 'w') as f:
                f.write(self.buildJson(True))
            if not os.path.exists(session_name):
                os.makedirs(session_name)
            os.chdir(session_path + "/" + session_id)
            if self.modules.a3d_api is not None and self.modules.vtk2model:
                a3d_model = self.modules.a3d_api.A3DModel(session_name) 
                a3d_model.read_json(session_path + "/" + session_id + "/sim_data_card.json")
                a3d_model.write_job_files() 
                a3d_model.submit_thermal_analysis(venue=self.venue.value, cpus=self.cores.value, wallTime=self.walltime.value) 
                a3d_model.submit_mechanical_analysis(venue=self.venue.value, cpus=self.cores.value, wallTime=self.walltime.value) 
                a3d_model.submit_odb2vtk(venue=self.venue.value)
                self.modules.vtk2model.processVTK(a3d_model.job_name, [400, 400, 400], "STATUS", 1.0) 
            
    def submitEnd(self, s):
        with self.debug:
            os.chdir(self.path)

    def rU(self, value, unit):
        return value
    
    def tU(self, value, unit):
        return value
    
a = Wizard()
a